In [2]:
import pandas as pd
import numpy as np
import sklearn
import zipfile
import time
import datetime
from sklearn import ensemble

In [3]:
z = zipfile.ZipFile('train.csv.zip')
data = pd.read_csv(z.open('train.csv'))
data = data.drop(['Descript', 'Resolution', 'Address'], axis=1)

z = zipfile.ZipFile('test.csv.zip')
test = pd.read_csv(z.open('test.csv'))
test = test.drop('Address', axis=1)

In [4]:
#Converts a formatted date/time string to a datetime object
#Adjust the formatting parameters to fit the input string

def strToDatetime(x):
    x = time.strptime(x, '%Y-%m-%d %X')
    x = datetime.datetime.fromtimestamp(time.mktime(x))  
    return x

In [5]:
#Convert the string date/time field to individual features

X = [d for d in data.Dates]
X = [strToDatetime(d) for d in X]

data["Month"]  = [d.month for d in X]
data["Day"] = [d.day for d in X]
data["Time"] = [d.time().hour*60 + d.time().minute for d in X]
data["Year"] = [d.year for d in X]

X = [d for d in test.Dates]
X = [strToDatetime(d) for d in X]

test["Month"]  = [d.month for d in X]
test["Day"] = [d.day for d in X]
test["Time"] = [d.time().hour*60 + d.time().minute for d in X]
test["Year"] = [d.year for d in X]

In [6]:
#Encode the categorical features

days = pd.get_dummies(data['DayOfWeek'])
neighborhoods = pd.get_dummies(data['PdDistrict'], prefix = 'Dist')
data = data.drop(['Dates','DayOfWeek', 'PdDistrict'], axis = 1)
data = data.join([neighborhoods, days])

days = pd.get_dummies(test['DayOfWeek'])
neighborhoods = pd.get_dummies(test['PdDistrict'], prefix = 'Dist')
test = test.drop(['Dates','DayOfWeek', 'PdDistrict'], axis = 1)
test = test.join([neighborhoods, days])

In [8]:
#Try a simple random fores on the data

clf = sklearn.ensemble.RandomForestClassifier()
clf.fit(data.drop("Category", axis =1), data.Category)
pred = clf.predict(test.drop("Id", axis = 1))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
#Read in the sample submission and copy its formatting into a new dataframe

z = zipfile.ZipFile('sampleSubmission.csv.zip')
sample = pd.read_csv(z.open('sampleSubmission.csv'))
toWrite = pd.DataFrame(0, index=np.arange(len(pred)), columns=sample.columns)

In [81]:
#Put the predictions in the output dataframe
for i in range(len(pred)):
    toWrite.ix[i]['Id'] = i
    toWrite.ix[i][pred[i]] = 1

In [85]:
toWrite.to_csv('predictions.csv', index=False)